In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import trange
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
NDVI_PATH = "../data/PROCESSED/ndvi.csv"
PROD_PATH = "../data/PROCESSED/manhuacu.csv"

In [ ]:
# MLP Hyperparameters
MLP_BATCH_SIZE = 16
MLP_WINDOW_SIZE = 10

# LSTM Hyperparameters
LSTM_WINDOW_SIZE = 10
LSTM_HIDDEN_SIZE = 64
LSTM_NUM_LAYERS = 1
LSTM_DROPOUT = 0.2
LSTM_EPOCHS = 400
LSTM_BATCH_SIZE = 16

# Computation
LSTM_DROPOUT = LSTM_DROPOUT if LSTM_NUM_LAYERS > 1 else 0

In [ ]:
def get_day_of_year_index(date: datetime):
    """Convert date to day of year."""
    return datetime(date.year, date.month, date.day).timetuple().tm_yday - 1


def get_sin_cos(x: float):
    """Convert x to sin and cos."""
    rad = 2 * np.pi * x
    return (np.sin(rad), np.cos(rad))


def encode_date(date: datetime):
    is_leap_year = 1 if date.year % 4 == 0 else 0
    total_year_days = 366 if is_leap_year else 365

    day_index = get_day_of_year_index(date)
    
    frac = day_index / total_year_days
    return get_sin_cos(frac)

# Test
print("Encoding date 2020-01-01")
print(encode_date(datetime(2020, 1, 1)))  # (0.0, 1.0)
print("\n")
print("Encoding date 2020-06-01")
print(encode_date(datetime(2020, 6, 1)))  # (0.5, 0.0)
print("\n")
print("Encoding date 2020-12-31")
print(encode_date(datetime(2020, 12, 31)))  # (0.9999999999999999, 1.0)
print("\n")

## 1. Carregar e Pré-processar Dados

### 1.1. Carregar e pre-processar os Dados

In [ ]:
NDVI = pd.read_csv(NDVI_PATH)
NDVI

In [ ]:

NDVI["N_Observations"] = NDVI.groupby("Year")["Data"].transform("count")

NDVI[["Date_sin", "Date_cos"]] = NDVI["Data"].apply(
    lambda x: pd.Series(encode_date(datetime.strptime(x, "%Y-%m-%d")))
)

# Assert order by Data (ascending)
NDVI = NDVI.sort_values(by="Data", ascending=True)

NDVI = NDVI[(NDVI["Year"] >= 2000) & (NDVI["Year"] <= 2023)]

NDVI

In [ ]:
PROD = pd.read_csv(PROD_PATH)
PROD = PROD[(PROD["Year"] >= 2000) & (PROD["Year"] <= 2023)]
# max_productivity = PROD["Productivity (kg/ha)"].max()
# PROD["Normalized_productivity"] = PROD["Productivity (kg/ha)"] / max_productivity
PROD

### 1.2. Visualizar os dados

In [ ]:
NDVI.plot(x="Data", y="NDVI", title="NDVI over time")

In [ ]:
# class MLPDataset(torch.utils.data.Dataset):
#     def __init__(self, ndvi_df, prod_df):
#         self.ndvi_df = ndvi_df
#         self.prod_df = prod_df

#     def __len__(self):
#         return self.ndvi_df["Year"].nunique()

#     def __getitem__(self, idx):
#         years = self.ndvi_df["Year"].sort_values().unique()
#         if idx >= len(years):
#             raise IndexError("Index out of range")
#         year = years[idx]
#         ndvi = self.ndvi_df[self.ndvi_df["Year"] == year]["NDVI"].values
#         prod = self.prod_df[self.prod_df["Year"] == year][
#             "Productivity (kg/ha)"
#         ].values[0]
#         return torch.tensor(ndvi, dtype=torch.float32), torch.tensor(
#             prod, dtype=torch.float32
#         )


# dataset = MLPDataset(NDVI_last_20_per_year, PROD)
# dataset[0]

# train_size = len(dataset) - 8
# valid_size = 4
# test_size = 4
# train_dataset = torch.utils.data.Subset(dataset, range(train_size))
# valid_dateset = torch.utils.data.Subset(dataset, range(train_size, train_size + valid_size))
# test_dataset = torch.utils.data.Subset(dataset, range(train_size + valid_size, train_size + valid_size + test_size))

### 2.2. Preparar Datasets

#### 2.1.1. Normalização

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalizer dados NDVI

NDVI["Year_norm"] = NDVI["Year"].copy()

ndvi_scaler = StandardScaler().fit(NDVI[["NDVI", "Year"]].values)
NDVI[["NDVI_norm", "Year_norm"]] = ndvi_scaler.transform(
    NDVI[["NDVI", "Year"]].values
)

NDVI


In [ ]:
# Normalizar produtividade
PROD["Year_norm"] = NDVI["Year"].copy()

prod_scaler = StandardScaler().fit(PROD[["Productivity (kg/ha)", "Year"]].values)
PROD[["Productivity_norm", "Year_norm"]] = prod_scaler.transform(
    PROD[["Productivity (kg/ha)", "Year"]].values
)
PROD

In [ ]:
class DatasetYearOfLast(torch.utils.data.Dataset):
    """
    DatasetYearOfLast - Dataset para previsão de produtividade

    X: Sequências de tamanho <WINDOW_SIZE> de observações de NDVI consecutivas (normalizado -1 a +1)
    y: Produtividade no Ano da última observação

    Features:
    - Sequências de NDVI (Já vem normalizado entre 0 e 1 da fonte)
    - Sequências de dia do ano com codificação circular no formato de Tupla: (Seno, Cosseno)
    - Sequência de ano da observação normalizado por z-score

    Label:
    - Produtividade (kg/ha) normalizada por z-score, relativa ao ano da última observação
    """

    def __init__(self, ndvi_df, prod_df, window_size=LSTM_WINDOW_SIZE):
        self.ndvi_df = ndvi_df
        self.prod_df = prod_df
        self.window_size = window_size

    def __len__(self):
        return len(self.ndvi_df) - self.window_size

    def __getitem__(self, idx):
        ndvi = self.ndvi_df.iloc[idx : idx + self.window_size][
            ["NDVI", "Date_sin", "Date_cos", "Year_norm"]
        ].values

        year = self.ndvi_df.iloc[idx + self.window_size - 1]["Year"]
        prod = self.prod_df[self.prod_df["Year"] == year]["Productivity_norm"].values[0]

        return torch.tensor(ndvi, dtype=torch.float32), torch.tensor(
            prod, dtype=torch.float32
        )


# Test Dataset
train_dataset_year_of_last = DatasetYearOfLast(
    NDVI[NDVI["Year"] <= 2016], PROD, LSTM_WINDOW_SIZE
)
validation_dataset_year_of_last = DatasetYearOfLast(
    NDVI[(NDVI["Year"] > 2016) & (NDVI["Year"] <= 2020)], PROD, LSTM_WINDOW_SIZE
)
test_dataset_year_of_last = DatasetYearOfLast(
    NDVI[NDVI["Year"] > 2020], PROD, LSTM_WINDOW_SIZE
)

print(validation_dataset_year_of_last[0][0].shape)
validation_dataset_year_of_last[0]

In [ ]:
class DatasetWeightedAverage(torch.utils.data.Dataset):
    """DatasetWeightedAverage - Dataset para previsão de produtividade

    X: Sequências de tamanho <WINDOW_SIZE> de observações de NDVI consecutivas (normalizado -1 a +1)
    y: Produtividade média ponderada entre a produtividade do ano da primeira observação e do ano da última observação
    - (normalizado por z-score)
    - A média é ponderada pela quantidade de observações do ano da primeira e do ano da última observação

    Features:
    - Sequências de NDVI (Já vem normalizado entre 0 e 1 da fonte)
    - Sequências de dia do ano com codificação circular no formato de Tupla: (Seno, Cosseno)
    - Sequência de ano da observação normalizado por z-score

    Label:
    - Produtividade (kg/ha) média ponderada entre o ano da primeira e do ano da última observação, normalizada por z-score
    """

    def __init__(self, ndvi_df, prod_df, window_size=LSTM_WINDOW_SIZE):
        self.ndvi_df = ndvi_df
        self.prod_df = prod_df
        self.window_size = window_size

    def __len__(self):
        return len(self.ndvi_df) - self.window_size

    def __getitem__(self, idx):
        ndvi = self.ndvi_df.iloc[idx : idx + self.window_size][
            ["NDVI", "Date_sin", "Date_cos", "Year_norm"]
        ].values

        year_first = self.ndvi_df.iloc[idx]["Year"]
        year_last = self.ndvi_df.iloc[idx + self.window_size]["Year"]

        prod_first = self.prod_df[self.prod_df["Year"] == year_first][
            "Productivity_norm"
        ].values[0]
        prod_last = self.prod_df[self.prod_df["Year"] == year_last][
            "Productivity_norm"
        ].values[0]

        n_obs_first = self.ndvi_df.iloc[idx : idx + self.window_size].loc[
            self.ndvi_df.iloc[idx : idx + self.window_size]["Year"] == year_first
        ].shape[0]
        n_obs_last = self.ndvi_df.iloc[idx : idx + self.window_size].loc[
            self.ndvi_df["Year"] == year_last
        ].shape[0]

        # Ponderação
        prod = (n_obs_first * prod_first + n_obs_last * prod_last) / (
            n_obs_first + n_obs_last
        )
        return torch.tensor(ndvi, dtype=torch.float32), torch.tensor(
            prod, dtype=torch.float32
        )
        
# Test Dataset
train_dataset_weighted_average = DatasetWeightedAverage(NDVI[NDVI["Year"] <= 2016], PROD, LSTM_WINDOW_SIZE)
validation_dataset_weighted_average = DatasetWeightedAverage(NDVI[(NDVI["Year"] > 2016) & (NDVI["Year"] <= 2020)], PROD, LSTM_WINDOW_SIZE)
test_dataset_weighted_average = DatasetWeightedAverage(NDVI[NDVI["Year"] > 2020], PROD, LSTM_WINDOW_SIZE)

validation_dataset_weighted_average[10]

## 3. Model training

### 3.1. Multi-layer Perceptron

Essa rede é uma feedforward perceptron multi-layer comum (1 camada interna).

As entradas são os 20 últimos NDVIs do ano, a saída é a produtividade prevista (kg/ha).

In [ ]:
mlp_network = nn.Sequential(
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(MLP_WINDOW_SIZE * 4, 256),
    nn.ReLU(),
    nn.Linear(256, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Tanh(),
)

def init_linear_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0.0, std=0.01)
        if m.bias is not None:
            nn.init.normal_(m.bias, mean=0.0, std=0.01)

mlp_network.apply(init_linear_weights)

# for name, param in mlp_network.named_parameters():
#     print(f"{name}: {param}")

# Step-by-step debug the MLP
# x = torch.randn(20, 4)
# print(f"Input shape: {x.shape}\n{x}\n")
# for i, layer in enumerate(mlp_network):
#     x = layer(x)
#     print(f"After layer {i} ({layer.__class__.__name__}): {x.shape}\n{x}\n")

In [ ]:
mlp_network = mlp_network.to(device)
optimizer = optim.Adam(mlp_network.parameters(), lr=5e-6)
loss_fn = nn.MSELoss()

mlp_losses = []
best_loss = float("inf")
saved_epoch = 0

train_loader_weighted_average = torch.utils.data.DataLoader(
    train_dataset_weighted_average, batch_size=MLP_BATCH_SIZE, shuffle=True
)
validation_loader_weighted_average = torch.utils.data.DataLoader(
    validation_dataset_weighted_average, batch_size=4, shuffle=True
)

for i in trange(400):
    mlp_network.train()
    for ndvi, prod in train_loader_weighted_average:
        ndvi, prod = ndvi.to(device), prod.to(device)
        optimizer.zero_grad()
        pred = mlp_network(ndvi)
        loss = loss_fn(pred.squeeze(dim=1), prod)
        loss.backward()
        optimizer.step()

    epoch_losses = []
    mlp_network.eval()
    with torch.no_grad():
        for ndvi, prod in validation_loader_weighted_average:
            ndvi, prod = ndvi.to(device), prod.to(device)
            pred = mlp_network(ndvi)
            loss = loss_fn(pred.squeeze(dim=1), prod)
            epoch_losses.append(loss.item())

        if np.mean(epoch_losses) < best_loss:
            best_loss = np.mean(epoch_losses)
            saved_epoch = i + 1
            torch.save(mlp_network.state_dict(), "mlp.pth")

    mlp_losses.append(np.mean(epoch_losses))
print(f"\n\nSaved MLP model\tepoch: {saved_epoch}\tvalidation loss: {best_loss:.4f}")

In [ ]:
def plot_loss(losses):
    """
    Plots the training loss over epochs.

    Args:
        losses (list): List of loss values for each epoch.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(losses, label="Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Over Epochs")
    plt.legend()
    plt.grid(True)
    plt.show()
    
plot_loss(mlp_losses)

### 3.2. LSTM

In [ ]:
import torch.nn as nn
import torch.optim as optim
import numpy as np


# Define model with Linear layer
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, dropout=LSTM_DROPOUT
        )
        self.fc = nn.Linear(hidden_size, 1)  # Output a single value

    def forward(self, x, hidden_n=None, hidden_c=None):
        if hidden_n is None or hidden_c is None:
            out, _ = self.lstm(x)
            return self.fc(out[:, -1, :])  # Get output of last time step
        else:
            out, (hidden_n, hidden_c) = self.lstm(x, (hidden_n, hidden_c))
            out = self.fc(out[:, -1, :])  # Get output of last time step
            return out, (hidden_n, hidden_c)


train_loader_weighted_average = torch.utils.data.DataLoader(
    train_dataset_weighted_average,
    batch_size=LSTM_BATCH_SIZE,
    shuffle=False,
    drop_last=True,
)
validation_loader_weighted_average = torch.utils.data.DataLoader(
    validation_dataset_weighted_average,
    batch_size=LSTM_BATCH_SIZE,
    shuffle=False,
    drop_last=True,
)


lstm_model = LSTMRegressor(
    input_size=4, hidden_size=LSTM_HIDDEN_SIZE, num_layers=LSTM_NUM_LAYERS
).to(device)

# for name, param in lstm_model.named_parameters():
#     print(f"{name}: {param}")


def init_lstm_weights(m):
    if isinstance(m, nn.LSTM):
        nn.init.xavier_uniform_(m.weight_ih_l0)
        nn.init.xavier_uniform_(m.weight_hh_l0)


lstm_model.apply(init_lstm_weights)
lstm_model.apply(init_linear_weights)

optimizer = optim.Adam(lstm_model.parameters(), lr=5e-4)
loss_fn = nn.MSELoss()

lstm_losses = []
best_loss = float("inf")
saved_epoch = 0


for i in range(LSTM_EPOCHS):
    h_n = torch.zeros(LSTM_NUM_LAYERS, LSTM_BATCH_SIZE, LSTM_HIDDEN_SIZE).to(
        device
    )  # Hidden state
    h_c = torch.zeros(LSTM_NUM_LAYERS, LSTM_BATCH_SIZE, LSTM_HIDDEN_SIZE).to(
        device
    )  # Cell state

    lstm_model.train()
    for ndvi, prod in train_loader_weighted_average:
        ndvi, prod = ndvi.to(device), prod.to(device)
        optimizer.zero_grad()
        pred, (h_n, h_c) = lstm_model(
            ndvi, h_n.detach(), h_c.detach()
        )
        last_pred = pred[:, -1]
        loss = loss_fn(last_pred, prod)
        loss.backward()
        nn.utils.clip_grad_norm_(lstm_model.parameters(), 1.0)
        optimizer.step()

    epoch_losses = []
    lstm_model.eval()
    with torch.no_grad():
        for ndvi, prod in validation_loader_weighted_average:
            ndvi, prod = ndvi.to(device), prod.to(device)
            pred = lstm_model(ndvi)
            last_pred = pred[:, -1]  # Get the last prediction
            loss = loss_fn(last_pred, prod)
            epoch_losses.append(loss.item())

        avg_loss = np.mean(epoch_losses)
        if avg_loss < best_loss:
            best_loss = avg_loss
            saved_epoch = i + 1
            torch.save(lstm_model.state_dict(), "lstm.pth")

    # if (i) % 10 == 0:
    print(f"Epoch {i+1}/{LSTM_EPOCHS} - Loss: {avg_loss:.4f}")
    lstm_losses.append(avg_loss)

print(f"\n\nSaved LSTM model\tepoch: {saved_epoch}\tvalidation loss: {best_loss:.4f}")

In [ ]:
plot_loss(lstm_losses)

## 4. Avaliação do Modelo

In [ ]:
test_loader_weighted_average = torch.utils.data.DataLoader(
    test_dataset_weighted_average, batch_size=1, shuffle=False
)

losses_mlp, pred_mlp = [], []
losses_lstm, pred_lstm = [], []
for ndvi, prod in test_loader_weighted_average:
    for model, losses_arr, preds_arr in zip(
        [mlp_network, lstm_model], [losses_mlp, losses_lstm], [pred_mlp, pred_lstm]
    ):
        model.eval()
        with torch.no_grad():
            ndvi, prod = ndvi.to(device), prod.to(device)
            pred = model(ndvi)
            last_pred = pred[:, -1]  # Get the last prediction
            loss = loss_fn(last_pred, prod)
            losses_arr.append(loss.item())
            preds_arr.append(last_pred.cpu().numpy()[0])

In [ ]:
np.mean(losses_mlp)

In [ ]:
np.mean(losses_lstm)

In [ ]:
test_dataset_weighted_average[0]

seq = []
for i in range(len(test_dataset_weighted_average)):
    _, prod = test_dataset_weighted_average[i]
    # print(f"Prod z-score: {prod}")
    # print("Predicted MLP z-score: ", pred_mlp[i])
    # print("Predicted LSTM z-score: ", pred_lstm[i])
    seq.append(
        {
            "Prod": prod.item(),
            "Pred_MLP": pred_mlp[i],
            "Pred_LSTM": pred_lstm[i],
        }
    )

df = pd.DataFrame(seq)
df

In [ ]:
df.plot(
    figsize=(12, 6),
    y=["Prod", "Pred_MLP", "Pred_LSTM"],
    title="Comparison of Prod, Pred_MLP, and Pred_LSTM",
    xlabel="Index",
    ylabel="Values",
    grid=True,
)
plt.legend(["Prod", "Pred_MLP", "Pred_LSTM"])
plt.tight_layout()
plt.show()
